In [1]:
import torch
from torch import nn

In [48]:
a = torch.tensor((127.48, -40.1, 0., 89.74, 124.38, -39.1, 126.48, 21.2, -35.99, 124.16,
   5.92,  41.68,  23.6,  -26.4,  -21.51, -20.6,   94.49,  85.07,  70.11 , 76.91))
a, a.shape

(tensor([127.4800, -40.1000,   0.0000,  89.7400, 124.3800, -39.1000, 126.4800,
          21.2000, -35.9900, 124.1600,   5.9200,  41.6800,  23.6000, -26.4000,
         -21.5100, -20.6000,  94.4900,  85.0700,  70.1100,  76.9100]),
 torch.Size([20]))

In [49]:
def asymmetric_quant(x: torch.Tensor, n: int):
    alpha = torch.max(x)
    beta = torch.min(x)
    s = (alpha - beta) / (2**n - 1)
    z = torch.round(-1 * (beta / s))
    xq = torch.round(x/s) + z
    xq = torch.clamp(xq, 0, 2**n -1)
    
    xf = s * (xq - z)
    
    return xq, xf
    # return xq

In [50]:
b, c = asymmetric_quant(a, 8)
b, b.shape, c, c.shape

(tensor([255.,   0.,  61., 198., 250.,   2., 253.,  93.,   6., 250.,  70., 124.,
          97.,  21.,  28.,  30., 205., 190., 168., 178.]),
 torch.Size([20]),
 tensor([127.4922, -40.0878,   0.0000,  90.0332, 124.2064, -38.7734, 126.1779,
          21.0296, -36.1447, 124.2064,   5.9146,  41.4021,  23.6584, -26.2871,
         -21.6868, -20.3725,  94.6334,  84.7758,  70.3179,  76.8896]),
 torch.Size([20]))

In [58]:
def loss(x: torch.Tensor, y: torch.Tensor):
    z = torch.mean((x-y)**2)
    return z.type_as(x)

In [59]:
# gg = torch.tensor((1, 2), dtype=torch.float)
# ff = torch.tensor((4, 5), dtype=torch.float)
hh = loss(a, c)
hh

tensor(0.0348)

In [60]:
def symmetric_quant(x: torch.Tensor, n: int):
    alpha = torch.max(torch.abs(x))
    s = alpha / (2**(n-1) -1)
    xq = torch.round(x / s)
    xq = torch.clamp(xq, -(2**(n-1) - 1), 2**(n-1) -1)
    
    xf = s * xq
    
    return xq, xf

In [61]:
x, y = symmetric_quant(a, 8)
x, y

(tensor([127., -40.,   0.,  89., 124., -39., 126.,  21., -36., 124.,   6.,  42.,
          24., -26., -21., -21.,  94.,  85.,  70.,  77.]),
 tensor([127.4800, -40.1512,   0.0000,  89.3364, 124.4687, -39.1474, 126.4762,
          21.0794, -36.1361, 124.4687,   6.0227,  42.1587,  24.0907, -26.0983,
         -21.0794, -21.0794,  94.3553,  85.3213,  70.2646,  77.2910]))

In [62]:
loss(a, y)

tensor(0.0772)

In [63]:
a = torch.randn(32, 512, 8, 32)
b = torch.randn(64, 256)
c = a * b
c.shape

RuntimeError: The size of tensor a (32) must match the size of tensor b (256) at non-singleton dimension 3

In [69]:
# import torch
# import torch.nn as nn
# 
# class LinformerSelfAttention(nn.Module):
#     def __init__(self, embed_dim, seq_len, num_heads, k):
#         """
#         Linformer Self-Attention Module.
#         
#         Args:
#             embed_dim (int): Dimension of the embeddings.
#             seq_len (int): Input sequence length.
#             num_heads (int): Number of attention heads.
#             k (int): Dimension to project the sequence length (low-rank approximation).
#         """
#         super().__init__()
#         assert embed_dim % num_heads == 0, "Embedding dimension must be divisible by number of heads."
#         
#         self.embed_dim = embed_dim
#         self.seq_len = seq_len
#         self.num_heads = num_heads
#         self.k = k
#         self.head_dim = embed_dim // num_heads
# 
#         # Projections for Query, Key, Value
#         self.query_proj = nn.Linear(embed_dim, embed_dim)
#         self.key_proj = nn.Linear(embed_dim, embed_dim)
#         self.value_proj = nn.Linear(embed_dim, embed_dim)
# 
#         # Low-rank approximation matrix
#         self.proj_matrix = nn.Parameter(torch.randn(seq_len, k))  # Learnable projection matrix
#         
#         # Final output projection
#         self.out_proj = nn.Linear(embed_dim, embed_dim)
# 
#     def forward(self, x):
#         """
#         Forward pass of Linformer Self-Attention.
#         
#         Args:
#             x (torch.Tensor): Input tensor of shape (batch_size, seq_len, embed_dim).
#         
#         Returns:
#             torch.Tensor: Output tensor of shape (batch_size, seq_len, embed_dim).
#         """
#         batch_size, seq_len, embed_dim = x.shape
#         print(f"Input shape: {x.shape}")  # Debugging: Input dimensions
#         
#         # Project input to Query, Key, Value
#         Q = self.query_proj(x)  # (batch_size, seq_len, embed_dim)
#         K = self.key_proj(x)    # (batch_size, seq_len, embed_dim)
#         V = self.value_proj(x)  # (batch_size, seq_len, embed_dim)
#         print(f"Query shape: {Q.shape}, Key shape: {K.shape}, Value shape: {V.shape}")
#         
#         # Reshape for multi-head attention
#         Q = Q.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)
#         K = K.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)
#         V = V.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)
#         print(f"Reshaped Query shape: {Q.shape}, Reshaped Key shape: {K.shape}, Reshaped Value shape: {V.shape}")
#         
#         # Project sequence length of K and V using the projection matrix
#         K_proj = torch.matmul(self.proj_matrix.T, K.transpose(2, 3))  # (batch_size, num_heads, k, head_dim)
#         print("projection")
#         print(self.proj_matrix.shape, self.proj_matrix.T.shape, K.transpose(2, 3).shape)
#         V_proj = torch.matmul(self.proj_matrix.T, V.transpose(2, 3))  # (batch_size, num_heads, k, head_dim)
#         print(f"Projected Key shape: {K_proj.shape}, Projected Value shape: {V_proj.shape}")
#         
#         # Compute scaled dot-product attention
#         attention_scores = torch.matmul(Q, K_proj.transpose(-1, -2)) / (self.head_dim ** 0.5)  # (batch_size, num_heads, seq_len, k)
#         attention_weights = torch.softmax(attention_scores, dim=-1)          # (batch_size, num_heads, seq_len, k)
#         print(f"Attention scores shape: {attention_scores.shape}, Attention weights shape: {attention_weights.shape}")
#         
#         # Weighted sum of values
#         attention_output = torch.matmul(attention_weights, V_proj)          # (batch_size, num_heads, seq_len, head_dim)
#         print(f"Attention output shape (per head): {attention_output.shape}")
#         
#         # Concatenate heads and project
#         attention_output = attention_output.transpose(1, 2).contiguous().view(batch_size, seq_len, embed_dim)
#         print(f"Concatenated output shape: {attention_output.shape}")
#         
#         output = self.out_proj(attention_output)  # (batch_size, seq_len, embed_dim)
#         print(f"Output shape after final projection: {output.shape}")
#         
#         return output
# 
# # Test the Linformer Self-Attention
# batch_size = 2
# seq_len = 16
# embed_dim = 64
# num_heads = 4
# k = 8
# 
# x = torch.rand(batch_size, seq_len, embed_dim)  # Dummy input
# linformer = LinformerSelfAttention(embed_dim, seq_len, num_heads, k)
# output = linformer(x)


Input shape: torch.Size([2, 16, 64])
Query shape: torch.Size([2, 16, 64]), Key shape: torch.Size([2, 16, 64]), Value shape: torch.Size([2, 16, 64])
Reshaped Query shape: torch.Size([2, 4, 16, 16]), Reshaped Key shape: torch.Size([2, 4, 16, 16]), Reshaped Value shape: torch.Size([2, 4, 16, 16])
projection
torch.Size([16, 8]) torch.Size([8, 16]) torch.Size([2, 4, 16, 16])
Projected Key shape: torch.Size([2, 4, 8, 16]), Projected Value shape: torch.Size([2, 4, 8, 16])
Attention scores shape: torch.Size([2, 4, 16, 8]), Attention weights shape: torch.Size([2, 4, 16, 8])
Attention output shape (per head): torch.Size([2, 4, 16, 16])
Concatenated output shape: torch.Size([2, 16, 64])
Output shape after final projection: torch.Size([2, 16, 64])


In [75]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

class a(nn.Module):
    def __init__(self):
        super().__init__()
        self.m1 = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.m1(x)
        return x
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'    

data = datasets.MNIST("/Users/mokshagrawal/Documents/machine-learning/quantization/root", transform=transforms.ToTensor(), train=True, download=True)
loader = DataLoader(data, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

model = a().to(device)
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=3e-4)

for e in range(20):
    model.train()
    loss_sum = 0
    loop = tqdm(enumerate(loader))
    for batch_idx, (x, y) in loop:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        loss_sum += loss.item()
    loss_sum /= len(loader)    
    print(f'epoch: {e+1}/ loss: {loss_sum:.4f}')    

for e in range(3):
    model.eval()
    c = 0
    t = 0
    with torch.no_grad():
        for batch_i, (x, y) in tqdm(enumerate(loader)):
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            _, just = torch.max(y_pred, dim=1)
            t += y.size(0)
            c += (just == y).sum().item()
        print(f'epoch: {e+1}/ acc: {c/t *100:.2f}%')

torch.save(model.state_dict(), 'model.pth')
print(os.path.getsize('model.pth')/1e3)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4525120.41it/s]


Extracting /Users/mokshagrawal/Documents/machine-learning/quantization/root/MNIST/raw/train-images-idx3-ubyte.gz to /Users/mokshagrawal/Documents/machine-learning/quantization/root/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 101748.19it/s]


Extracting /Users/mokshagrawal/Documents/machine-learning/quantization/root/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/mokshagrawal/Documents/machine-learning/quantization/root/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

epoch: 1/ acc: 99.93%



938it [00:06, 153.87it/s]

epoch: 2/ acc: 99.93%



938it [00:08, 114.59it/s]


epoch: 3/ acc: 99.93%
2145.588


In [76]:
def train(model):
    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(model.parameters(), lr=3e-4)
    for e in range(20):
        model.train()
        loss_sum = 0
        loop = tqdm(enumerate(loader))
        for batch_idx, (x, y) in loop:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            loss_sum += loss.item()
        loss_sum /= len(loader)    
        print(f'epoch: {e+1}/ loss: {loss_sum:.4f}')  
        
def eval(model):
    for e in range(3):
        model.eval()
        c = 0
        t = 0
        with torch.no_grad():
            for batch_i, (x, y) in tqdm(enumerate(loader)):
                x, y = x.to(device), y.to(device)
                y_pred = model(x)
                _, just = torch.max(y_pred, dim=1)
                t += y.size(0)
                c += (just == y).sum().item()
            print(f'epoch: {e+1}/ acc: {c/t *100:.2f}%')        

In [78]:
class aq(nn.Module):
    def __init__(self):
        super().__init__()
        self.o1 = torch.quantization.QuantStub()
        self.m1 = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
        self.o2 = torch.quantization.DeQuantStub()
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.o1(x)
        x = self.m1(x)
        x = self.o2(x)
        return x

model2 = aq().to(device)  
model2.load_state_dict(model.state_dict())
model2.eval()

model2.qconfig = torch.ao.quantization.default_qconfig
model2 = torch.ao.quantization.prepare(model2)
print(model2)

for e in range(3):
    model2.eval()
    c = 0
    t = 0
    with torch.no_grad():
        for batch_i, (x, y) in tqdm(enumerate(loader)):
            x, y = x.to(device), y.to(device)
            y_pred = model2(x)
            _, just = torch.max(y_pred, dim=1)
            t += y.size(0)
            c += (just == y).sum().item()
        print(f'epoch: {e+1}/ acc: {c/t *100:.2f}%')

print(model2)        
model2 = torch.ao.quantization.convert(model2)
print(model2)
# print(torch.int_repr(model2.linear1.weight()))
# print(torch.dequantize(model2.linear1.weight()))
# torch.save(model2.state_dict(), 'model2.pth')
# print(os.path.getsize('model2.pth')/1e3)
# train(model2)

aq(
  (o1): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (m1): Sequential(
    (0): Linear(
      in_features=784, out_features=512, bias=True
      (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
    )
    (1): ReLU()
    (2): Linear(
      in_features=512, out_features=256, bias=True
      (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
    )
    (3): ReLU()
    (4): Linear(
      in_features=256, out_features=10, bias=True
      (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
    )
  )
  (o2): DeQuantStub()
)


938it [00:08, 113.47it/s]

epoch: 1/ acc: 99.93%



938it [00:14, 65.81it/s] 

epoch: 2/ acc: 99.93%



938it [00:13, 71.49it/s] 


epoch: 3/ acc: 99.93%
aq(
  (o1): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=0.0, max_val=1.0)
  )
  (m1): Sequential(
    (0): Linear(
      in_features=784, out_features=512, bias=True
      (activation_post_process): MinMaxObserver(min_val=-10.2401123046875, max_val=7.791010856628418)
    )
    (1): ReLU()
    (2): Linear(
      in_features=512, out_features=256, bias=True
      (activation_post_process): MinMaxObserver(min_val=-14.952836036682129, max_val=15.126261711120605)
    )
    (3): ReLU()
    (4): Linear(
      in_features=256, out_features=10, bias=True
      (activation_post_process): MinMaxObserver(min_val=-74.66053009033203, max_val=48.738040924072266)
    )
  )
  (o2): DeQuantStub()
)


RuntimeError: Didn't find engine for operation quantized::linear_prepack NoQEngine

In [79]:
torch.save(model2.state_dict(), 'model2.pth')
print(os.path.getsize('model2.pth')/1e3)

2149.066


In [83]:
import torch.nn.quantized as nnq
from torch.quantization import get_default_qconfig, prepare, convert
torch.backends.quantized.engine = 'qnnpack'
model2 = convert(model2)
print(model2)

aq(
  (o1): Quantize(scale=tensor([0.0079]), zero_point=tensor([0]), dtype=torch.quint8)
  (m1): Sequential(
    (0): QuantizedLinear(in_features=784, out_features=512, scale=0.14197735488414764, zero_point=72, qscheme=torch.per_tensor_affine)
    (1): ReLU()
    (2): QuantizedLinear(in_features=512, out_features=256, scale=0.2368432879447937, zero_point=63, qscheme=torch.per_tensor_affine)
    (3): ReLU()
    (4): QuantizedLinear(in_features=256, out_features=10, scale=0.9716423153877258, zero_point=77, qscheme=torch.per_tensor_affine)
  )
  (o2): DeQuantize()
)


In [93]:
print(torch.int_repr(model2.m1[0].weight()))
print(torch.dequantize(model2.m1[0].weight()))
torch.save(model2.state_dict(), 'model2.pth')
print(os.path.getsize('model2.pth')/1e3)
# train(model2)

tensor([[  3,   7,  10,  ...,  -2,   7,  -6],
        [  6,  -5, -11,  ...,   5,   4,   6],
        [  1,  -1,   3,  ...,   8,  -1,  -8],
        ...,
        [ -2,  -4,   8,  ...,  -6,  -8,   8],
        [ 10,  -3,   1,  ...,   4,  -5,  -2],
        [  3,   4,   6,  ...,  -6,   9,   5]], dtype=torch.int8)
tensor([[ 0.0094,  0.0219,  0.0313,  ..., -0.0063,  0.0219, -0.0188],
        [ 0.0188, -0.0157, -0.0345,  ...,  0.0157,  0.0125,  0.0188],
        [ 0.0031, -0.0031,  0.0094,  ...,  0.0251, -0.0031, -0.0251],
        ...,
        [-0.0063, -0.0125,  0.0251,  ..., -0.0188, -0.0251,  0.0251],
        [ 0.0313, -0.0094,  0.0031,  ...,  0.0125, -0.0157, -0.0063],
        [ 0.0094,  0.0125,  0.0188,  ..., -0.0188,  0.0282,  0.0157]])
542.874


In [94]:
for e in range(3):
    model2.eval()
    c = 0
    t = 0
    with torch.no_grad():
        for batch_i, (x, y) in tqdm(enumerate(loader)):
            x, y = x.to(device), y.to(device)
            y_pred = model2(x)
            _, just = torch.max(y_pred, dim=1)
            t += y.size(0)
            c += (just == y).sum().item()
        print(f'epoch: {e+1}/ acc: {c/t *100:.2f}%')

938it [00:06, 136.13it/s]

epoch: 1/ acc: 99.92%



938it [00:05, 167.86it/s]

epoch: 2/ acc: 99.92%



938it [00:05, 178.65it/s]

epoch: 3/ acc: 99.92%


In [95]:
class qat(nn.Module):
    def __init__(self):
        super().__init__()
        self.o1 = torch.quantization.QuantStub()
        self.m1 = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
        self.o2 = torch.quantization.DeQuantStub()
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.o1(x)
        x = self.m1(x)
        x = self.o2(x)
        return x
    

model3 = qat().to(device)

model3.qconfig = torch.ao.quantization.default_qconfig
model3.train()
model3 = torch.ao.quantization.prepare_qat(model3)
print(model3)

qat(
  (o1): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (m1): Sequential(
    (0): Linear(
      in_features=784, out_features=512, bias=True
      (weight_fake_quant): MinMaxObserver(min_val=inf, max_val=-inf)
      (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
    )
    (1): ReLU()
    (2): Linear(
      in_features=512, out_features=256, bias=True
      (weight_fake_quant): MinMaxObserver(min_val=inf, max_val=-inf)
      (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
    )
    (3): ReLU()
    (4): Linear(
      in_features=256, out_features=10, bias=True
      (weight_fake_quant): MinMaxObserver(min_val=inf, max_val=-inf)
      (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
    )
  )
  (o2): DeQuantStub()
)


In [96]:
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model3.parameters(), lr=3e-4)

for e in range(1):
    # model3.train()
    loss_sum = 0
    loop = tqdm(enumerate(loader))
    for batch_idx, (x, y) in loop:
        x, y = x.to(device), y.to(device)
        y_pred = model3(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        loss_sum += loss.item()
    loss_sum /= len(loader)    
    print(f'epoch: {e+1}/ loss: {loss_sum:.4f}')   

938it [00:10, 89.31it/s] 

epoch: 1/ loss: 0.3581


In [97]:
print(model3)

qat(
  (o1): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=0.0, max_val=1.0)
  )
  (m1): Sequential(
    (0): Linear(
      in_features=784, out_features=512, bias=True
      (weight_fake_quant): MinMaxObserver(min_val=-0.15922757983207703, max_val=0.11339440196752548)
      (activation_post_process): MinMaxObserver(min_val=-5.275387763977051, max_val=5.154850482940674)
    )
    (1): ReLU()
    (2): Linear(
      in_features=512, out_features=256, bias=True
      (weight_fake_quant): MinMaxObserver(min_val=-0.16703025996685028, max_val=0.16207362711429596)
      (activation_post_process): MinMaxObserver(min_val=-8.715311050415039, max_val=9.392265319824219)
    )
    (3): ReLU()
    (4): Linear(
      in_features=256, out_features=10, bias=True
      (weight_fake_quant): MinMaxObserver(min_val=-0.13709521293640137, max_val=0.11249437183141708)
      (activation_post_process): MinMaxObserver(min_val=-18.374143600463867, max_val=18.19473648071289)
    )
  )
  (o2): De

In [98]:
model3.eval()


model3 = torch.ao.quantization.convert(model3)
print(model3)

qat(
  (o1): Quantize(scale=tensor([0.0079]), zero_point=tensor([0]), dtype=torch.quint8)
  (m1): Sequential(
    (0): QuantizedLinear(in_features=784, out_features=512, scale=0.08212786167860031, zero_point=64, qscheme=torch.per_tensor_affine)
    (1): ReLU()
    (2): QuantizedLinear(in_features=512, out_features=256, scale=0.1425793468952179, zero_point=61, qscheme=torch.per_tensor_affine)
    (3): ReLU()
    (4): QuantizedLinear(in_features=256, out_features=10, scale=0.28794392943382263, zero_point=64, qscheme=torch.per_tensor_affine)
  )
  (o2): DeQuantize()
)


In [99]:
print(torch.int_repr(model3.m1[0].weight()))

tensor([[ 16,  20,  -6,  ...,   1, -28, -16],
        [-24,   8,  -2,  ..., -17,  21,  22],
        [ 23,   0,  23,  ..., -20,  -7,  13],
        ...,
        [-25,  -7,  27,  ...,  -2, -20,  23],
        [  0,  25,  -3,  ..., -19,  16,  -8],
        [-23,  -3,  17,  ...,  14,  -3,  -3]], dtype=torch.int8)


In [100]:
for e in range(3):
    # model2.eval()
    c = 0
    t = 0
    with torch.no_grad():
        for batch_i, (x, y) in tqdm(enumerate(loader)):
            x, y = x.to(device), y.to(device)
            y_pred = model3(x)
            _, just = torch.max(y_pred, dim=1)
            t += y.size(0)
            c += (just == y).sum().item()
        print(f'epoch: {e+1}/ acc: {c/t *100:.2f}%')

938it [00:06, 153.00it/s]

epoch: 1/ acc: 95.03%



938it [00:05, 177.81it/s]

epoch: 2/ acc: 95.03%



938it [00:05, 173.51it/s]

epoch: 3/ acc: 95.03%
